In [11]:
# needed for API
import pandas as pd
import numpy as np
import datetime as dt
import pandas_datareader as pdr
import yfinance as yfin
yfin.pdr_override()
from dotenv import load_dotenv
import os
import json
import requests

def get_econ_data ():
    # define data for DataReader
    end = dt.date.today()
    start= end - dt.timedelta(days=365*21)
    start_str = start.strftime('%Y-%m-%d')
    end_str = end.strftime('%Y-%m-%d')

    # select tables, enter as dataframe
    list_of_pct_tables = ['SOFR30DAYAVG', 'SOFR', 'EFFR', 'AAA', 'DBAA', 'T10YIE', 'T5YIE', 'MORTGAGE30US', 'DGS30', 'DGS1','BAMLH0A0HYM2EY']
    # AAA - BBB is spread
    lisf_of_val_tables = ['SOFRVOL', 'RECPROUSM156N', 'SAHMREALTIME']
    econ_df_pct = pdr.DataReader(list_of_pct_tables, 'fred', start_str, end_str)
    econ_df_val = pdr.DataReader(lisf_of_val_tables, 'fred', start_str, end_str)
    
    #filling blank values with prior value
    econ_df_pct.fillna(method='ffill', inplace=True)
    econ_df_val.fillna(method='ffill', inplace=True)
    
    #create spread metrics
    spread_df = pd.DataFrame()
    spread_df['ab_bond_spread'] = econ_df_pct["DBAA"] - econ_df_pct["AAA"]
    spread_df['junk_bond_spread'] = econ_df_pct["BAMLH0A0HYM2EY"] - econ_df_pct["AAA"]
    spread_df['int_spread'] = econ_df_pct["DGS30"] - econ_df_pct["DGS1"]
    
    # calculate daily changes
    temp_df = econ_df_val[['RECPROUSM156N','SAHMREALTIME']]
    econ_df_pct = econ_df_pct.diff()
    econ_df_val = econ_df_val.pct_change()
    
    # add back in unchange probability
    econ_df_val[['RECPROUSM156N','SAHMREALTIME']] = temp_df
    
    # combine data
    econ_df = pd.concat([econ_df_pct, econ_df_val, spread_df], axis=1)
    
    # remove inf values
    econ_df[np.isinf(econ_df)] = 0
    # fill blanks
    temp_df = econ_df[['RECPROUSM156N','SAHMREALTIME']]
    temp_df.fillna(method='ffill', inplace=True)
    econ_df[['RECPROUSM156N','SAHMREALTIME']] = temp_df
    
    #drop blank columns
    econ_df = econ_df.dropna(subset='EFFR')
    
    # update index to date
    econ_df.index = econ_df.index.date

    return econ_df

C:\Users\micha\AppData\Local\Temp\ipykernel_15600\2606458185.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.fillna(method='ffill', inplace=True)


,SOFR30DAYAVG,SOFR,EFFR,AAA,DBAA,T10YIE,T5YIE,MORTGAGE30US,DGS30,DGS1,BAMLH0A0HYM2EY,SOFRVOL,RECPROUSM156N,SAHMREALTIME,ab_bond_spread,junk_bond_spread,int_spread
2002-04-17,NaN,NaN,0.05,NaN,0.02,NaN,NaN,NaN,0.07,-0.03,-0.04,NaN,NaN,NaN,NaN,NaN,3.42
2002-04-18,NaN,NaN,-0.02,NaN,0.02,NaN,NaN,NaN,0.01,-0.03,-0.03,NaN,NaN,NaN,NaN,NaN,3.46
2002-04-19,NaN,NaN,-0.04,NaN,-0.04,NaN,NaN,NaN,-0.06,-0.01,0.00,NaN,NaN,NaN,NaN,NaN,3.41
2002-04-22,NaN,NaN,0.06,NaN,-0.05,NaN,NaN,0.00,-0.02,0.02,0.02,NaN,NaN,NaN,NaN,NaN,3.37
2002-04-23,NaN,NaN,-0.02,NaN,0.00,NaN,NaN,0.00,0.00,0.00,-0.01,NaN,NaN,NaN,NaN,NaN,3.37
2002-04-24,NaN,NaN,0.05,NaN,-0.04,NaN,NaN,0.00,-0.06,-0.06,-0.04,NaN,NaN,NaN,NaN,NaN,3.37
2002-04-25,NaN,NaN,0.06,NaN,0.01,NaN,NaN,0.00,0.00,-0.02,0.00,NaN,NaN,NaN,NaN,NaN,3.39
2002-04-26,NaN,NaN,-0.03,NaN,-0.02,NaN,NaN,-0.06,0.01,-0.02,-0.01,NaN,NaN,NaN,NaN,NaN,3.42
2002-04-29,NaN,NaN,0.02,NaN,0.04,NaN,NaN,0.00,0.02,0.05,-0.02,NaN,NaN,NaN,NaN,NaN,3.39
2002-04-30,NaN,NaN,0.01,NaN,0.00,NaN,NaN,0.00,-0.03,-0.01,-0.28,NaN,NaN,NaN,NaN,NaN,3.37
